# Bibliotecas

In [1]:
import argparse
import configparser
import pandas as pd
from pathlib import Path
import os.path as osp
from tqdm.auto import tqdm

# from schwebsearch import load_sch, SCHWebSearch, parse_result_file, save_annotation_file

import configparser
import json
import os
import pandas as pd
import re
import requests
import string
import uuid


from collections import Counter
from datetime import datetime, timedelta
from nltk.corpus import stopwords
from pathlib import Path
from unicodedata import normalize
from urllib.parse import urlparse

In [2]:
# time format for json result files
RESULT_TS_FORMAT = '%Y%m%d%H%M%S%f'
# time format for wordcloud files
ANNOTATION_TS_FORMAT = '%d/%m/%Y %H:%M:%S'
ANOTATION_FILE_TS_FORMAT = '%Y.%m.%d_T%H.%M.%S'

# Desenvolvimento

In [3]:
# args to desenv
total_itens_to_query=5
grace_period=180
verbose=True

In [4]:
# read config file
config = configparser.ConfigParser()
if osp.exists('websearch_config.ini'):
    print('Reading config file ... ', end='')
    try:
        config.read('websearch_config.ini')
        sch_database_file = Path(config['SCHWEBSEARCH']['sch_database_file'])
        search_results_folder = Path(config['SCHWEBSEARCH']['search_results_folder'])
        parsed_results_folder = Path(config['SCHWEBSEARCH']['parsed_results_folder'])
        annotation_folder = Path(config['SCHWEBSEARCH']['annotation_folder'])
        print('success.')
    except:
        print('config file is corrupted. Execution aborted.')
        exit(-1)
else:
    print('Config file not found. Execution aborted.')

Reading config file ... success.


In [5]:
search_results_folder, parsed_results_folder

(WindowsPath('D:/Datasets/SCHWebSearch/search_results'),
 WindowsPath('D:/Datasets/SCHWebSearch/parsed_search_results'))

In [6]:
def parse_result_file(file, parsed_results_folder=None, max_words=25):

    parse_url = lambda url: '.'.join(urlparse(url).netloc.split('.')[-3:])
    result_id = str(uuid.uuid4())

    EMPTY_RESULT = {'ID': result_id, 
                    'DataHora': None,
                    'Computador': None,
                    'Usuário': None, 
                    'Homologação': None, 
                    'Atributo': None,
                    'Valor': None,  
                    'Situação': -1}

    if isinstance(file, str):
        file = Path(file)
    
    search_date, search_engine, search_term, _ = re.split('[_.]',file.name)
    search_site = None
    
    try:
        with open(file) as f:
            results = json.load(f)
    except:
        return EMPTY_RESULT

    try:
        with open('blacklisted_sites.txt') as f:
            blacklisted_sites = f.readlines()
            blacklisted_sites = [site.strip() for site in blacklisted_sites]
    except:
        blacklisted_sites = None
        
    lines = []
    if search_engine == 'GOOGLE':
        # results without items in keys are empty
        if 'items' in results.keys():
            for item in results['items']:
                search_site = item['displayLink']
                if is_black_listed_site(search_site):
                    continue
                else:
                    search_site = parse_url(search_site)
                    lines.append(item['title'])
                    if 'snippet' in item.keys():
                        lines.append(item['snippet'])
        else:
            return EMPTY_RESULT
                        
    elif search_engine == 'BING':
        # results without webPages in keys are empty
        if 'webPages' in results.keys():
            for item in results['webPages']['value'][:10]:
                search_site = item['url']
                if is_black_listed_site(search_site):
                    continue
                else:
                    search_site = parse_url(search_site)
                    lines.append(item['name'])
                    if 'snippet' in item.keys():
                        lines.append(item['snippet'])
        else:
            return EMPTY_RESULT
                 
    if len(lines) >= 1:
        words = tokenizer(' '.join(lines))
        words_conter = Counter(words)
        wordCloud_dict = {key:value for key,value in words_conter.most_common(max_words)}
        wordCloud_json = json.dumps(wordCloud_dict, ensure_ascii=False)
        situacao = 1
    else:
        wordCloud_json = ''
        situacao = -1
        
    wourdCloudInfo_dict = {
        'metaData': {
            'Version': 1,
            'Source': search_engine,
            'Mode': 'API',
            'Fields': ['Name', 'Snippet'],
            'nWords': max_words
        },
        'searchedWord': search_term,
        'cloudOfWords': wordCloud_json
        }
    
    wourdCloudInfo_json = json.dumps(wourdCloudInfo_dict, ensure_ascii=False)
    
    return {'ID': result_id,
            'DataHora': datetime.strptime(search_date,RESULT_TS_FORMAT).strftime(ANNOTATION_TS_FORMAT),
            'Computador': os.environ['COMPUTERNAME'],
            'Usuário': os.environ['USERNAME'], 
            'Homologação': f'{search_term[:5]}-{search_term[5:7]}-{search_term[-5:]}', 
            'Atributo': 'WordCloud',
            'Valor': wourdCloudInfo_json,  
            'Situação': situacao}

In [7]:
file=r'D:\Datasets\SCHWebSearch\search_results\20240606090127364095_GOOGLE_057771703345.json'
parse_result_file(file)

{'ID': '54be3f17-ed1c-4006-a559-bdee6cc8cdae',
 'DataHora': None,
 'Computador': None,
 'Usuário': None,
 'Homologação': None,
 'Atributo': None,
 'Valor': None,
 'Situação': -1}